In [42]:
import os
import numpy as np
import tensorflow as tf
from collections import Counter
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Image Labels fetching

In [26]:
# Define your data directory containing superhero class folder
data_dir = "D:\Git\HeroClassifier\Images"

# Initialize empty lists to store image paths and labels
images = []
labels = []

# Iterate through each superhero folder
for class_dir in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_dir)
    
    # Iterate through all images in the each folders
    for filename in os.listdir(class_path):
        image_path = os.path.join(class_path, filename)
        images.append(image_path)
        labels.append(class_dir)
        
Counter(labels)        

Counter({'Batman': 155,
         'Black Panther': 155,
         'Black Widow': 154,
         'Captain America': 154,
         'Hulk': 154,
         'Iron Man': 155,
         'Spiderman': 154,
         'Superman': 155,
         'The Flash': 154,
         'Wonder Woman': 155})

#### Train Test Split (Train,Validation,Test = 70,10,20) 

In [27]:
test_size = 0.2
val_size = 0.1

# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=8980)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=8980)

In [28]:
Counter(y_train)

Counter({'Captain America': 117,
         'Batman': 108,
         'Wonder Woman': 107,
         'Black Panther': 107,
         'Spiderman': 112,
         'Iron Man': 106,
         'Black Widow': 113,
         'The Flash': 113,
         'Superman': 118,
         'Hulk': 111})

#### Data Preprocessing

In [29]:
def preprocess_image(img_path, img_size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    return img_array

In [30]:
img_size = (224, 224) 

X_train_processed = [preprocess_image(img, img_size) for img in X_train]
X_val_processed = [preprocess_image(img, img_size) for img in X_val]
X_test_processed = [preprocess_image(img, img_size) for img in X_test]

In [31]:
# One hot encoding

label_mapping = {label: idx for idx, label in enumerate(set(labels))}
num_classes = len(label_mapping)

y_train_encoded = tf.keras.utils.to_categorical([label_mapping[label] for label in y_train], num_classes=num_classes)
y_val_encoded = tf.keras.utils.to_categorical([label_mapping[label] for label in y_val], num_classes=num_classes)
y_test_encoded = tf.keras.utils.to_categorical([label_mapping[label] for label in y_test], num_classes=num_classes)

#### Transfer Learning - selecting VGG16 as base model

In [32]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#### CNN model using VGG16, ReLU, Dropout, Adam

In [33]:
# Model over VGG16
model = models.Sequential()
model.add(base_model)  # VGG16 trained on imagenet as base
model.add(layers.Flatten()) 
model.add(layers.Dense(512, activation='relu')) # Activation: ReLU
model.add(BatchNormalization())                 # BatchNormalization
model.add(layers.Dropout(0.5))                  # Dropout 50%
model.add(layers.Dense(num_classes, activation='softmax')) # Softmax for multiclass classification

In [39]:
# Load saved weights
model.load_weights('superhero_classifier_model_3.h5')  

In [35]:
# Loss function: Crossentropy, Optimizer: Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

#### Data Augmentation

In [36]:
# Data Augmentation for the training data
train_datagen = ImageDataGenerator(rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

train_generator = train_datagen.flow(np.array(X_train_processed), y_train_encoded, batch_size=32)

In [37]:
epochs = 5
batch_size = 32

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_generator
                    ,steps_per_epoch=len(X_train_processed)//batch_size
                    ,epochs=epochs
                    ,validation_data=(np.array(X_val_processed), y_val_encoded)
                    ,callbacks=[early_stopping]
                   )

Epoch 1/5
34/34 [==============================] - 367s 11s/step - loss: 0.1584 - accuracy: 0.9444 - val_loss: 1.2843 - val_accuracy: 0.7581
Epoch 2/5
34/34 [==============================] - 458s 13s/step - loss: 0.1162 - accuracy: 0.9713 - val_loss: 0.9398 - val_accuracy: 0.8145
Epoch 3/5
34/34 [==============================] - 424s 12s/step - loss: 0.1200 - accuracy: 0.9651 - val_loss: 1.3532 - val_accuracy: 0.7581
Epoch 4/5
34/34 [==============================] - 380s 11s/step - loss: 0.1145 - accuracy: 0.9574 - val_loss: 1.2641 - val_accuracy: 0.7823
Epoch 5/5
34/34 [==============================] - 369s 11s/step - loss: 0.1298 - accuracy: 0.9574 - val_loss: 0.7877 - val_accuracy: 0.8306


In [40]:
# Evaluation Matrices
test_loss, test_acc = model.evaluate(np.array(X_test_processed), y_test_encoded)
print(f"Test Accuracy: {test_acc}")

10/10 [==============================] - 21s 2s/step - loss: 0.4947 - accuracy: 0.8867
Test Accuracy: 0.8867313861846924


In [22]:
# Save the model for deployment if required
model.save("superhero_classifier_model_3.h5")

In [ ]:
# superhero_classifier_model_2 - 78.64 Test
# superhero_classifier_model_3 - 88.67 Test